Imports

In [9]:
import numpy as np
import matplotlib as plt
import scipy as sp 
import pandas as pd

#classification
from sklearn.neighbors import KNeighborsClassifier #k-plus proches voisins

#Partitionnement
from sklearn.cluster import AgglomerativeClustering #Regroupement hiérarchique (Paritionnement binaire)
#from pyclustering.cluster.kmedoids import kmedoids

#réduction de dimensionnalité
from sklearn.decomposition import KernelPCA #ce n'est pas PCoA mais on peut l'utiliser pour que le résultat soit le même
from sklearn.manifold import Isomap

DataSet Download and Cleanup

In [ ]:
#--------------------------------ADULT--------------------------------#

class DataParser:
  def __init__(self):
    data_features_db = pd.read_csv('adult.csv').to_numpy()
    self.data_features = np.delete(data_features_db,0,0)

  def splitData(self):
    features = self.data_features
    size = len(features)

    training_features = []
    test_features = []


    for i in range(size):
      if i%5==1:
        test_features.append(features[i])
  
      else:
        training_features.append(features[i])

    

    training_x = np.array(training_features)
    test_x = np.array(test_features)

    return (training_x, test_x)


Similarity

In [7]:
#--------------------------------ADULT--------------------------------#
# 0, age: num
# 1, workclass: cat
# 2, fnlwgt: num QUE PEDO CON ESTA
# 3, education: cat
# 4, education-num: num
# 5, marital-status: cat
# 6, occupation: cat
# 7, relationship: cat
# 8, race: cat
# 9, sex: cat
# 10, capital-gain: num
# 11, capital-loss: num
# 12, hours-per-week: num
# 13, native-country: cat
# 14, class, cat
def adult_dissimilarity(x, y, num_avg_dist):
  numeric_feats_x = x[0,4,10,11,12]
  numeric_feats_y = y[0,4,10,11,12]
  #distance euclidienne sur features numériques
  num_dissimilarity = np.linalg.norm(numeric_feats_x - numeric_feats_y)
  #estimation du poids pour les features catégoriques
  individual_wgt = num_avg_dist/5.0

  

  categoric_feats_x = x[1,5,6,7,8,9,13,14]
  categoric_feats_y = y[1,5,6,7,8,9,13,14]
  cat_dissimilarity = 0
  for i in range (len(categoric_feats_x)):
    if categoric_feats_x == categoric_feats_y:
      cat_dissimilarity += individual_wgt
  return num_dissimilarity + cat_dissimilarity
    

def adult_dissimilarity_matrix(X,Y):
  return adult_dissimilarity(X[:,:], Y[:,:])

#--------------------------------MNIST--------------------------------#

  


  



Isomap

In [ ]:
isomap = Isomap(n_components=1, n_neighbors=2, metric='precomputed')
isomap_circle = isomap.fit_transform(circle_cosine_dissimilarity)
isomap_infinity = isomap.transform(infinity_circle_cosine_dissimilarity)

PCoA

In [ ]:
pcoa = KernelPCA(n_components=1, kernel='precomputed')
pcoa_circle = pcoa.fit_transform(-.5*circle_cosine_dissimilarity**2) #-.5*D**2 est crucial!!!
pcoa_infinity = pcoa.transform(-.5*infinity_circle_cosine_dissimilarity**2) #-.5*D**2 est crucial!!!

K-Médoids

In [ ]:
initial_medoids = [0,1,2]
kmedoids_instance = kmedoids(circle_cosine_dissimilarity, initial_medoids, data_type='distance_matrix')
kmedoids_instance.process() #training

kmedoids_circle = kmedoids_instance.predict(circle_cosine_dissimilarity)
kmedoids_infinity = kmedoids_instance.predict(infinity_circle_cosine_dissimilarity)

Partitionnement binaire

In [ ]:
def agglomerative_clustering_predict(agglomerative_clustering, dissimilarity_matrix):
    average_dissimilarity = list()
    for i in range(agglomerative_clustering.n_clusters):
        ith_clusters_dissimilarity = dissimilarity_matrix[:, np.where(agglomerative_clustering.labels_==i)[0]]
        average_dissimilarity.append(ith_clusters_dissimilarity.mean(axis=1))
    return np.argmin(np.stack(average_dissimilarity), axis=0)

agglomerative_clustering = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='average')
agglomerative_clustering.fit(circle_cosine_dissimilarity)

agglo_circle = agglomerative_clustering_predict(agglomerative_clustering, circle_cosine_dissimilarity)
agglo_infinity = agglomerative_clustering_predict(agglomerative_clustering, infinity_circle_cosine_dissimilarity)

KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2, metric='precomputed', algorithm='brute')
knn.fit(circle_cosine_dissimilarity, quadrants)

knn_circle = knn.predict(circle_cosine_dissimilarity)
knn_infinity = knn.predict(infinity_circle_cosine_dissimilarity)